In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.metrics import classification_report
import scipy as stats


In [2]:
df=pd.read_csv('/home/camilo/MLOps/MLOPs/data/adult.csv')

In [8]:
df

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,Private,103497,Some-college,10,Never-married,Prof-specialty,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [4]:
df.isnull().sum()

age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital-status     0
occupation         0
relationship       0
race               0
gender             0
capital-gain       0
capital-loss       0
hours-per-week     0
native-country     0
income             0
dtype: int64

In [5]:
round((df.isin(['?']).sum() / df.shape[0]) * 100, 2).astype(str) + ' %'

age                 0.0 %
workclass          5.73 %
fnlwgt              0.0 %
education           0.0 %
educational-num     0.0 %
marital-status      0.0 %
occupation         5.75 %
relationship        0.0 %
race                0.0 %
gender              0.0 %
capital-gain        0.0 %
capital-loss        0.0 %
hours-per-week      0.0 %
native-country     1.75 %
income              0.0 %
dtype: object

In [6]:
df=df.replace("?",np.nan)

In [ ]:
round((df.isnull().sum() / df.shape[0]) * 100, 2).astype(str) + ' %'

In [ ]:
income =df['income'].value_counts(normalize=True)
round(income * 100, 2).astype('str') + ' %'

In [9]:
columns_with_nan = ['workclass', 'occupation', 'native-country']
for col in columns_with_nan:
    df[col].fillna(df[col].mode()[0], inplace = True) 

In [ ]:
df.corr()

In [10]:
for col in df.columns:
  if df[col].dtypes == 'object':         
    encoder = LabelEncoder()         
    df[col] = encoder.fit_transform(df[col])

In [11]:
X=df.iloc[:,0:14].values
y=df.iloc[:,-1].values

In [12]:
from sklearn.ensemble import ExtraTreesClassifier
selector = ExtraTreesClassifier(random_state = 42)
selector.fit(X, y)
feature_imp = selector.feature_importances_
for index, val in enumerate(feature_imp):
    print(index, round((val * 100), 2)) , 

0 15.75
1 3.88
2 17.64
3 3.78
4 8.34
5 7.27
6 7.05
7 9.01
8 1.41
9 2.96
10 8.9
11 2.89
12 9.58
13 1.54


In [ ]:
X = X.drop(['workclass', 'education', 'race', 'sex', 'capital.loss', 'native.country'], axis = 1)


In [ ]:
for col in X.columns:     
  scaler = StandardScaler()     
  X[col] = scaler.fit_transform(X[col].values.reshape(-1, 1))

In [ ]:
from imblearn.over_sampling import RandomOverSampler 
ros = RandomOverSampler(random_state = 42)
ros.fit(X, y)
X_resampled, y_resampled = ros.fit_resample(X, y)
round(y_resampled.value_counts(normalize=True) * 100, 2).astype('str') + ' %'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size = 0.2, random_state = 42)
print("X_train shape:", X_train.shape) 
print("X_test shape:", X_test.shape) 
print("Y_train shape:", y_train.shape) 
print("Y_test shape:", y_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier #creamos nuestro modelo, y ademas no elegimos nuestros hiper parametros
ran_for = RandomForestClassifier(random_state = 42)
ran_for.fit(X_train, y_train)
y_pred_ran_for = ran_for.predict(X_test)

In [ ]:
print('Random Forest Classifier:')
print('Accuracy score:',round(accuracy_score(y_test, y_pred_ran_for) * 100, 2))
print('F1 score:',round(f1_score(y_test, y_pred_ran_for) * 100, 2))

In [13]:
from sklearn.model_selection import cross_val_score

In [ ]:
score=cross_val_score(ran_for,X_train,y_train,cv=5, scoring="f1_macro")
print(f"el valor es {score} ")
print("Mean 5-Fold f1_macro: {}".format(np.mean(score)))

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True)
rf_reg = RandomForestClassifier()

In [ ]:
scores = []
for train_index,test_index in kf.split(X_train):
  X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
  

print('Scores from each Iteration: ', scores)
print('Average K-Fold Score :' , np.mean(scores))

In [ ]:
ran_for.get_params()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV # para hallar nuestros hiperparametros
n_estimators = [int(x) for x in np.linspace(start = 40, stop = 150, num = 15)] 
max_depth = [int(x) for x in np.linspace(start=40, stop=150, num = 15)]
max_samples=[int(x) for x in np.linspace(start=4,stop=150,num=15)]
param_dist = {
    'n_estimators' : n_estimators,
    'max_depth' : max_depth,
    'max_samples':max_samples
}
rf_tuned = RandomForestClassifier(random_state = 42)
rf_cv = RandomizedSearchCV(estimator = rf_tuned, param_distributions = param_dist, cv = 5, random_state = 42)
rf_cv.fit(X_train, y_train)
rf_cv.best_score_


In [ ]:
rf_cv.best_params_

In [ ]:
rf_best = RandomForestClassifier(max_depth = 63, n_estimators = 79,max_samples=129, random_state = 42)#aca uso los hiperparametros
rf_best.fit(X_train, y_train)
y_pred_rf_best = rf_best.predict(X_test)
print('Random Forest Classifier:') 
print('Accuracy score:',round(accuracy_score(y_test, y_pred_rf_best) * 100, 2)) 
print('F1 score:',round(f1_score(y_test, y_pred_rf_best) * 100, 2))

In [ ]:
cm = confusion_matrix(y_test, y_pred_rf_best)
sns.heatmap(cm,annot=True,fmt=".1%")

In [ ]:
print(classification_report(y_test, y_pred_rf_best))